In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

from config import *
from helpers import *

### Filter function for vulnerability commits

In [8]:
def filter_commits(commits):
    """ wip """
    label= []
    global STATS

    for index, commit in commits.iterrows():
        STATS["commits"] += 1

        if not filter_title(CONFIG, get_commit_title(commit)):
            label.append(0)
            continue
        if any(hit in get_commit_title(commit) for hit in ["Merge", "Revert"]):
            label.append(0)
            continue
        STATS["fixes"] += 1

        hits = filter_commit(CONFIG, commit)
        if not hits:
            label.append(0)
            continue
        STATS["filtered"] += 1

        reporter = get_commit_reporter(commit['commit_msg'])
        CVEs = get_commit_cves(commit['commit_msg']) # enough to not check title?

        if not filter_reporter(CONFIG, reporter):
            label.append(0)
            continue
        else:
            label.append(1)

    return label

### Load and Process dataset

In [9]:
df = pd.read_csv('data/bigvul_devign_cvefixes_neuralsentry_commits.csv')
df = df.drop_duplicates(subset=['commit_hash'],keep='first')
df = df.dropna(subset=['commit_msg', 'commit_hash'])
train_df, df = train_test_split(df, test_size=0.2, random_state=420)
    
linux_test_df = pd.read_csv('data/linux_test.csv')
vulnfix = linux_test_df[linux_test_df['labels'] == 1]
non_vulnfix = linux_test_df[linux_test_df['labels'] == 0]
linux_test_df = pd.concat([non_vulnfix, vulnfix])

### Prediction on BigVul, Devign, CVEfixes and NeuralSentry Dataset

In [10]:
predictions = filter_commits(df)
labels = df['labels'].tolist()
generate_metrics(labels, predictions)


>>> Confusion Matrix: 
TP: 953, FP: 168 
FN: 2602, TN: 3275 

>>> Accuracy:  0.6041726207487854 

>>>: Classification Report:
               precision    recall  f1-score   support

  non-bugfix       0.56      0.95      0.70      3443
      bugfix       0.85      0.27      0.41      3555

    accuracy                           0.60      6998
   macro avg       0.70      0.61      0.56      6998
weighted avg       0.71      0.60      0.55      6998



### Prediction on Linux Kernel CVEs Dataset

In [11]:
predictions = filter_commits(linux_test_df)
labels = linux_test_df['labels'].tolist()
generate_metrics(labels, predictions)


>>> Confusion Matrix: 
TP: 918, FP: 42 
FN: 909, TN: 461 

>>> Accuracy:  0.5918454935622317 

>>>: Classification Report:
               precision    recall  f1-score   support

  non-bugfix       0.34      0.92      0.49       503
      bugfix       0.96      0.50      0.66      1827

    accuracy                           0.59      2330
   macro avg       0.65      0.71      0.58      2330
weighted avg       0.82      0.59      0.62      2330

